<a href="https://colab.research.google.com/github/dianakang/DIANA_GIT/blob/master/kaggle_nlp_preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 확인하기

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"younjungkang","key":"add359cbb7542087839afc562d5313fc"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 64.9MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 18.0MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 52.6MB/s]


In [4]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import metrics

In [6]:
trn = pd.read_csv('/content/train.csv')
tst = pd.read_csv('/content/test.csv')

In [7]:
# 데이터 크기 확인
print(trn.shape, tst.shape)

(7613, 5) (3263, 4)


In [8]:
print(trn.info())
print("-----"*10)
print(tst.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB
None


In [9]:
trn.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
tst.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# trn 데이터 결측값 개수 확인
trn.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [12]:
# tst 데이터 결측값 개수 확인
tst.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

# 2. 전처리

In [13]:
# 데이터 전처리 -> tweet-preprocessor 이용
!pip install tweet-preprocessor

*   tweet-preprocessor

  *   URLs
  *   Mentions
  *   Reserved words (RT, FAV)
      *  RT: Retweet의 약자
      *  FAV:  Favorite의 약자
  *   Emojis
  *   Smileys
  *   JSON and .txt file support

In [14]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from collections import Counter
import string
import re
from nltk.corpus import wordnet as wn
import preprocessor 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
stop = set(STOPWORDS).union(set(['FAV' , 'RT']))  ## .union() =  합집합
lemma = WordNetLemmatizer()  ## NLTK에서는 표제어(단어의 뿌리) 추출을 위한 도구인 WordNetLemmatizer를 지원
preprocessor.set_options(preprocessor.OPT.URL, preprocessor.OPT.MENTION, preprocessor.OPT.NUMBER, preprocessor.OPT.RESERVED)  ## OPT = option

def clean(text):   
    text = preprocessor.clean(text)
    text = re.sub(r'[^\w\s]','',text)
    stop_free = " ".join([i for i in text.split(' ') if (i not in stop)])
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    return normalized

In [16]:
# 정제된 데이터
trn.text = trn.text.apply(clean)
tst.text = tst.text.apply(clean)

In [17]:
trn.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds Reason earthquake May ALLAH Forgive u,1
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1
2,5,NaN,NaN,All resident asked shelter place notified offi...,1
3,6,NaN,NaN,people receive wildfire evacuation order Calif...,1
4,7,NaN,NaN,Just got sent photo Ruby Alaska smoke wildfire...,1


In [18]:
tst.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened terrible car crash
1,2,NaN,NaN,Heard earthquake different city stay safe ever...
2,3,NaN,NaN,forest fire spot pond goose fleeing across str...
3,9,NaN,NaN,Apocalypse lighting Spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kill China Taiwan
